In [42]:
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
%matplotlib inline

# Acquiring data from Lending Club API

In [43]:
api_key = open("pzapikey.txt", "r").read()
header = {"Authorization" : api_key}
url = "https://api.lendingclub.com/api/investor/v1/loans/listing"

r = requests.get(url, headers=header)

In [44]:
data = r.json()
MyData = data["loans"]

In [45]:
with open("current_list.txt", "w") as output:
    json.dump(MyData, output)

In [46]:
df_current = pd.read_json("current_list.txt")

In [47]:
list(df_current.columns)

['accNowDelinq',
 'accOpenPast24Mths',
 'acceptD',
 'addrState',
 'addrZip',
 'allUtil',
 'annualInc',
 'annualIncJoint',
 'applicationType',
 'avgCurBal',
 'bcOpenToBuy',
 'bcUtil',
 'chargeoffWithin12Mths',
 'collections12MthsExMed',
 'creditPullD',
 'delinq2Yrs',
 'delinqAmnt',
 'desc',
 'disbursementMethod',
 'dti',
 'dtiJoint',
 'earliestCrLine',
 'empLength',
 'empTitle',
 'expD',
 'expDefaultRate',
 'ficoRangeHigh',
 'ficoRangeLow',
 'fundedAmount',
 'grade',
 'homeOwnership',
 'housingPayment',
 'iLUtil',
 'id',
 'ilsExpD',
 'initialListStatus',
 'inqFi',
 'inqLast12m',
 'inqLast6Mths',
 'installment',
 'intRate',
 'investorCount',
 'isIncV',
 'isIncVJoint',
 'listD',
 'loanAmount',
 'maxBalBc',
 'memberId',
 'moSinOldIlAcct',
 'moSinOldRevTlOp',
 'moSinRcntRevTlOp',
 'moSinRcntTl',
 'mortAcc',
 'mtgPayment',
 'mthsSinceLastDelinq',
 'mthsSinceLastMajorDerog',
 'mthsSinceLastRecord',
 'mthsSinceRcntIl',
 'mthsSinceRecentBc',
 'mthsSinceRecentBcDlq',
 'mthsSinceRecentInq',
 'mth

# Read historical data

In [8]:
df_2015 = pd.read_csv("2015data_11102018.csv", skiprows = 1)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df_2014 = pd.read_csv("2014data_11132018.csv", skiprows = 1)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### features comparison

In [24]:
df_current.columns.to_frame().to_csv("currentcolumnname.txt", header=None, index=None)

In [25]:
df_2015.columns.to_frame().to_csv("2015columnname.txt", header=None, index=None)

In [26]:
df_2014.columns.to_frame().to_csv("2014columnname.txt", header=None, index=None)

# Format column names

In [27]:
columnName2014 = df_2014.columns.values
columnName2015 = df_2015.columns.values
columnNameNow = df_current.columns.values

In [28]:
column_2015_also_in_2014 = []

for column in columnName2015:
    if column in columnName2014:
        column_2015_also_in_2014.append(column)

In [29]:
print(len(column_2015_also_in_2014),len(columnName2014))

145 145


In [30]:
columnName2015_mlt = columnName2015.copy()
columnName2014_mlt = columnName2014.copy()
columnNameNow_mlt = columnNameNow.copy()

In [31]:
for i in range(len(columnName2015_mlt)):
    columnName2015_mlt[i] = (columnName2015_mlt[i]).replace("_", "")
    
for i in range(len(columnName2014_mlt)):
    columnName2014_mlt[i] = (columnName2014_mlt[i]).replace("_", "")
    
for i in range(len(columnNameNow_mlt)):
    columnNameNow_mlt[i] = (columnNameNow_mlt[i]).lower()

In [32]:
for i in range(len(columnName2015_mlt)):
    columnName2015_mlt[i] = (columnName2015_mlt[i]).replace("amnt", "amount")
    
for i in range(len(columnNameNow_mlt)):
    columnNameNow_mlt[i] = (columnNameNow_mlt[i]).replace("amnt", "amount")
    
for i in range(len(columnNameNow_mlt)):
    columnNameNow_mlt[i] = (columnNameNow_mlt[i]).replace("addrzip", "zipcode")

for i in range(len(columnNameNow_mlt)):
    columnNameNow_mlt[i] = (columnNameNow_mlt[i]).replace("isincv", "verificationstatus")

In [33]:
pd.DataFrame(list((set(columnNameNow_mlt) ^ set(columnName2015_mlt)) & set(columnNameNow_mlt))).to_csv("commoncolumnname_currentunique.txt", header=None, index=None)
pd.DataFrame(list((set(columnNameNow_mlt) ^ set(columnName2015_mlt)) & set(columnName2015_mlt))).to_csv("commoncolumnname_2015unique.txt", header=None, index=None)
pd.DataFrame(list(set(columnNameNow_mlt) & set(columnName2015_mlt))).to_csv("commoncolumnname.txt", header=None, index=None)

In [34]:
df_2015.columns = list(columnName2015_mlt)
df_2014.columns = list(columnName2014_mlt)
df_current.columns = list(columnNameNow_mlt)
df_2015.drop(df_2015.tail(2).index,inplace=True)

##### Common features number

In [39]:
len(set(columnNameNow_mlt) & set(columnName2015_mlt))

106

In [40]:
commom_features = list(set(columnNameNow_mlt) & set(columnName2015_mlt))

In [49]:
pd.DataFrame(commom_features).to_csv("commonfeatures_new.txt", index=False, header=None)

##### Save cleaned data frame for historical data

In [25]:
df_2015_loan = df_2015[commom_features + ["issued", "loanstatus"]]

In [26]:
df_2015_loan.shape

(421095, 108)

In [28]:
df_2015_loan.to_csv("loan_2015.csv", index=False)

In [29]:
df_2015_loan

,percentbcgt75,mortacc,numactvbctl,term,openrv24m,memberid,secappopenacc,inqfi,openactil,ficorangelow,...,numrevaccts,inqlast6mths,totalilhighcreditlimit,totalbclimit,mthssincerecentrevoldelinq,bcutil,totcollamt,secappficorangehigh,issued,loanstatus
0,0.0,1.0,1.0,36 months,1.0,NaN,NaN,1.0,3.0,750.0,...,14.0,0.0,46367.0,12700.0,NaN,21.9,0.0,NaN,Dec-2015,Current
1,100.0,3.0,3.0,36 months,1.0,NaN,NaN,0.0,4.0,710.0,...,11.0,0.0,68648.0,4400.0,NaN,90.2,0.0,NaN,Dec-2015,Fully Paid
2,50.0,1.0,2.0,36 months,3.0,NaN,NaN,1.0,6.0,715.0,...,21.0,1.0,107522.0,20400.0,46.0,38.0,0.0,NaN,Dec-2015,Fully Paid
3,0.0,0.0,3.0,36 months,1.0,NaN,NaN,2.0,2.0,835.0,...,13.0,1.0,30017.0,76200.0,NaN,3.0,0.0,NaN,Dec-2015,Fully Paid
4,25.0,5.0,4.0,60 months,3.0,NaN,NaN,0.0,0.0,700.0,...,8.0,0.0,0.0,16300.0,26.0,62.7,0.0,NaN,Dec-2015,Current
5,0.0,2.0,2.0,60 months,1.0,NaN,NaN,1.0,1.0,740.0,...,13.0,1.0,33631.0,30500.0,NaN,10.9,0.0,NaN,Dec-2015,Current
6,0.0,0.0,2.0,36 months,2.0,NaN,NaN,1.0,3.0,695.0,...,7.0,0.0,29240.0,21000.0,27.0,52.3,0.0,NaN,Dec-2015,Charged Off
7,50.0,2.0,2.0,60 months,1.0,NaN,NaN,1.0,7.0,670.0,...,17.0,0.0,51664.0,5900.0,NaN,26.8,0.0,NaN,Dec-2015,Fully Paid
8,100.0,2.0,3.0,36 months,5.0,NaN,NaN,0.0,1.0,665.0,...,15.0,1.0,6000.0,8500.0,45.0,94.8,0.0,NaN,Dec-2015,Fully Paid
9,0.0,1.0,2.0,36 months,3.0,NaN,NaN,3.0,2.0,675.0,...,9.0,1.0,13734.0,2400.0,69.0,37.2,722.0,NaN,Dec-2015,Current


## Data acquiring finished